# Week 13: MLOps & Model Serving

Transitioning from research to production: APIs, Docker, and Monitoring.

## Learning Objectives
1. Build a FastAPI Model Server
2. Containerize with Docker (Conceptual)
3. Implement Logging and Monitoring middleware

In [ ]:
import time
from typing import Dict, Any
# Note: In a real notebook, you would install fastapi and uvicorn
# !pip install fastapi uvicorn

## 1. Model Wrapper Class

Standard interface for model inference.

In [ ]:
class ModelService:
    def __init__(self, model_path: str):
        self.model = self._load_model(model_path)
        self.initialized = True
    
    def _load_model(self, path: str):
        print(f"Loading model from {path}...")
        # Mock model loading
        return lambda x: {"prediction": "class_A", "confidence": 0.95}
    
    def predict(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        start_time = time.time()
        
        # Mock inference
        result = self.model(input_data)
        
        latency = (time.time() - start_time) * 1000
        result["latency_ms"] = round(latency, 2)
        return result

## 2. FastAPI Application (Mock)

Simulating how to structure the API.

In [ ]:
# This is pseudo-code for a notebook environment as APIs run in a server loop

app_structure = """
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

app = FastAPI(title="AI Model Service")
model_service = ModelService("model.pkl")

class PredictionRequest(BaseModel):
    text: str
    top_k: int = 3

@app.post("/predict")
async def predict(request: PredictionRequest):
    try:
        return model_service.predict(request.dict())
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health():
    return {"status": "healthy", "model_loaded": model_service.initialized}
"""

print("FastAPI App Structure:")
print(app_structure)

## 3. Monitoring & Logging

Implementing a custom logger for metrics.

In [ ]:
import json
import datetime

class MetricsLogger:
    def __init__(self, log_file="metrics.jsonl"):
        self.log_file = log_file
    
    def log_request(self, endpoint: str, status: int, latency: float):
        entry = {
            "timestamp": datetime.datetime.now().isoformat(),
            "endpoint": endpoint,
            "status_code": status,
            "latency_ms": latency
        }
        print(f"[LOG] {json.dumps(entry)}")
        # In real world: append to file or send to Prometheus/Datadog

# Test Logger
logger = MetricsLogger()
logger.log_request("/predict", 200, 45.2)

## 4. Dockerfile Template

Best practices for containerizing Python AI apps.

In [ ]:
dockerfile = """
# Use slim python image for smaller size
FROM python:3.10-slim

# Set working directory
WORKDIR /app

# Install dependencies first (caching layer)
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy code
COPY src/ ./src
COPY main.py .

# Expose port
EXPOSE 8000

# Run command
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
"""

print(dockerfile)